In [30]:
import pandas as pd
import os
import io
from PIL import Image

In [3]:
df = pd.read_csv('./text_detection/result/res_test2.txt',sep=',',names=['x1','y1','x2','y2','x3','y3','x4','y4'])

In [4]:
df['text_size'] = df['y3'] - df['y1']

In [7]:
df.head()

,x1,y1,x2,y2,x3,y3,x4,y4,text_size
0,150,183,213,183,213,208,150,208,25
1,221,183,290,183,290,208,221,208,25
2,298,183,361,183,361,208,298,208,25
3,372,183,416,183,416,208,372,208,25
4,424,183,490,183,490,208,424,208,25


줄 나누기

In [6]:
line_list = []
text_size = df['text_size'].mean()
start = len(line_list)
for _ in range(len(df)):
    x1,y1,x2,y2,x3,y3,x4,y4,text_size = df.loc[_]
    box = x1,y1,x2,y2,x3,y3,x4,y4
    
    idx = 0
    
    if _ ==0:
        li = []
        li.append(box)
        line_list.append(li)
            
    for line in line_list[start:]:
        do_append = 'F'
        if _ ==0:
            break
        for i in line:
            if (abs(((y1 + y3) /2) - ((i[1] + i[5])/2)) < text_size):
                line.append(box)
                line.sort()
                do_append = 'T'
                break    
        if do_append == 'T':
            break
        else:
            idx += 1

    if idx == len(line_list[start:]):
        new_line = []
        new_line.append(box)
        line_list.append(new_line)

단락 나누기

In [26]:
before_x_point = 0
before_y_point = 0
text_structure = []
idx = 0
for line in line_list:
    if idx ==0:
        new_paragraph = []
        new_paragraph.append(line)
    
    else:
        if (line[0][1] - before_y_point) > 1.5 *text_size:
            text_structure.append(new_paragraph)
            new_paragraph = []
            new_paragraph.append(line)
        else:
            new_paragraph.append(line)
         
    before_x_point = line[0][0]
    before_y_point = line[0][1]
         
    before_x_point = line[0][0]
    before_y_point = line[0][1]
    idx += 1
    
    if idx == len(line_list):
        text_structure.append(new_paragraph)

In [15]:
len(text_structure[0])

1

단락별 줄별 이미지 저장

In [20]:
img = Image.open('./text_detection/test/test2/test2.jpg','r')

In [27]:
img = Image.open('./text_detection/test/test2/test2.jpg','r')
a = 0
k = 0
for paragraph in text_structure:
    path = './text_detection/result/test2_structure/'
    paragraph_path = path + 'paragraph' + str(k) + '/'
    os.mkdir(paragraph_path)
    for line in paragraph:
        line.sort()
        line_path = paragraph_path + 'line' + str(a) + '/'
        os.mkdir(line_path)
        b = 0
        for box in line:
            x1,y1,x2,y2,x3,y3,x4,y4 = box
            image = img.crop((x1,y1,x3,y3))
            image.save(line_path + str(b) +'.jpg')
            b += 1
        a += 1
    k += 1